In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image

In [ ]:
def load_image_from_url(url):
    with urllib.request.urlopen(url) as url_response:
        image = np.array(Image.open(url_response))
    return image

In [ ]:
image = load_image_from_url("https://assets.8thlight.com/images/insights/posts/2022-03-25-what-is-a-convolution/fruits.jpg")
plt.imshow(image)
plt.show()

In [ ]:
# Convert image to grayscale
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define convolution kernels
left_sobel = np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
])

blur = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0]
]) / 5

identity = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
])

intensity = 5
sharpen = identity + (identity - blur) * intensity

# Apply convolutions
sobel_result = cv2.filter2D(image_grayscale, -1, left_sobel)
blur_result = cv2.filter2D(image_grayscale, -1, blur)
sharpen_result = cv2.filter2D(image_grayscale, -1, sharpen)

# Display results
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
axs[0].imshow(image_grayscale, cmap='gray')
axs[0].set_title('Grayscale')
axs[1].imshow(sobel_result, cmap='gray')
axs[1].set_title('Sobel Filter')
axs[2].imshow(blur_result, cmap='gray')
axs[2].set_title('Blur Filter')
axs[3].imshow(sharpen_result, cmap='gray')
axs[3].set_title('Sharpen Filter')

# Remove axis ticks for all subplots
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()


In [ ]:
# Create a figure with 4 subplots in one row
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

# Resize image
image_resized = cv2.resize(image, (0, 0), fx=0.1, fy=0.1)
axs[0].imshow(image_resized)
axs[0].set_title('Resized')

# Rotate image
image_rotated = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
axs[1].imshow(image_rotated)
axs[1].set_title('Rotated')

# Flip image
image_flipped = cv2.flip(image, 1)
axs[2].imshow(image_flipped)
axs[2].set_title('Flipped')

# Crop image
image_cropped = image[100:200, 100:200]
axs[3].imshow(image_cropped)
axs[3].set_title('Cropped')

# Remove axis ticks for all subplots
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
def non_maximum_suppression(magnitude, angle):
    # Convert angle to degrees
    angle = np.rad2deg(angle)
    
    # Normalize angle to be within 0-180 degrees
    angle[angle < 0] += 180
    
    # Create a grid of indices
    y, x = np.indices(magnitude.shape)

    # Create a grid of angles
    angles = np.zeros_like(magnitude)
    angles[angle < 22.5] = 0
    angles[(angle >= 22.5) & (angle < 67.5)] = 1
    angles[(angle >= 67.5) & (angle < 112.5)] = 2
    angles[(angle >= 112.5) & (angle < 157.5)] = 3
    angles[angle >= 157.5] = 0

    # Initialize the output array
    suppressed = np.zeros_like(magnitude)

    # Perform non-maximum suppression
    for i in range(1, magnitude.shape[0] - 1):
        for j in range(1, magnitude.shape[1] - 1):
            if angles[i, j] == 0:
                neighbors = [magnitude[i, j-1], magnitude[i, j+1]]
            elif angles[i, j] == 1:
                neighbors = [magnitude[i-1, j+1], magnitude[i+1, j-1]]
            elif angles[i, j] == 2:
                neighbors = [magnitude[i-1, j], magnitude[i+1, j]]
            else:
                neighbors = [magnitude[i-1, j-1], magnitude[i+1, j+1]]
            
            if magnitude[i, j] >= max(neighbors):
                suppressed[i, j] = magnitude[i, j]

    return suppressed


def double_thresholding(image):
    high_threshold = 70
    low_threshold = 35
    # Create output array initialized with zeros (weak edges)
    result = np.zeros_like(image)
    
    # Find strong edge pixels
    strong_edges = (image >= high_threshold)
    
    # Find weak edge pixels 
    weak_edges = (image >= low_threshold) & (image < high_threshold)
    
    # Set strong edges to 255
    result[strong_edges] = 255
    
    # Set weak edges to 128 (temporary value)
    result[weak_edges] = 128
    
    # Find all connected components
    num_labels, labels = cv2.connectedComponents(result.astype(np.uint8))
    
    # Iterate through all components
    for label in range(1, num_labels):
        component = (labels == label)
        if np.any(component & strong_edges):
            # If component contains any strong edge, keep all its pixels
            result[component] = 255
        else:
            # If component doesn't contain any strong edge, remove it
            result[component] = 0
    
    # Convert any remaining 128 values to 0
    result[result == 128] = 0
    
    return result
    
def canny_edge_detection(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise and detail
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Compute gradients using Sobel kernels
    grad_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute gradient magnitude and direction
    magnitude = cv2.magnitude(grad_x, grad_y)
    angle = cv2.phase(grad_x, grad_y, angleInDegrees=True)
    
    # Non-maximum suppression
    non_max_suppressed = non_maximum_suppression(magnitude, angle)
    
    # Double thresholding
    thresholded = double_thresholding(non_max_suppressed)
    
    return thresholded

In [ ]:
# Custom Canny Edge Detection
image_canny_custom = canny_edge_detection(image)

# OpenCV Canny Edge Detection
image_canny_opencv = cv2.Canny(image, 100, 200)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Display custom Canny edge detection result
ax1.imshow(image_canny_custom, cmap='gray')
ax1.set_title('Custom Canny Edge Detection')
ax1.axis('off')

# Display OpenCV Canny edge detection result
ax2.imshow(image_canny_opencv, cmap='gray')
ax2.set_title('OpenCV Canny Edge Detection')
ax2.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
image = load_image_from_url("https://vincmazet.github.io/bip/_images/roof.jpg")
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
plt.hist(image.ravel(), bins=256, range=(0, 256), color='black')
plt.title('Histogram of Intensity')
plt.xlabel('Intensity')
plt.ylabel('Frequency')

# Add text annotations to show which side is light and dark
plt.text(20, plt.gca().get_ylim()[1]*0.9, 'Dark', fontsize=12, ha='left', va='center')
plt.text(235, plt.gca().get_ylim()[1]*0.9, 'Light', fontsize=12, ha='right', va='center')

# Add arrows to emphasize the dark and light sides
plt.annotate('', xy=(0, plt.gca().get_ylim()[1]*0.85), xytext=(50, plt.gca().get_ylim()[1]*0.85),
             arrowprops=dict(arrowstyle='<-', color='gray'))
plt.annotate('', xy=(255, plt.gca().get_ylim()[1]*0.85), xytext=(205, plt.gca().get_ylim()[1]*0.85),
             arrowprops=dict(arrowstyle='<-', color='gray'))

plt.show()

In [ ]:
# Create a figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Simple thresholding
ret_simple, thresh_simple = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
ax1.imshow(thresh_simple, cmap='gray')
ax1.set_title('Simple Thresholding')
ax1.axis('off')

# Otsu thresholding
ret_otsu, thresh_otsu = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU)
ax2.imshow(thresh_otsu, cmap='gray')
ax2.set_title(f'Otsu Thresholding\nThreshold: {ret_otsu:.2f}')
ax2.axis('off')

# Adaptive thresholding
thresh_adaptive = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
ax3.imshow(thresh_adaptive, cmap='gray')
ax3.set_title('Adaptive Thresholding')
ax3.axis('off')

plt.tight_layout()
plt.show()